In [1]:
import os

os.chdir("../")
%pwd

'c:\\Users\\anfe1\\OneDrive\\Escritorio\\carpetaANFE\\AI_ML_Projects\\Text-Summarization'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_chkpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    per_device_eval_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [3]:
from textSummarization.constants import *
from textSummarization.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self) -> None:
        config_file_path = CONFIG_FILE_PATH
        params_file_path = PARAMS_FILE_PATH

        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])
    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.training_arguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir= config.root_dir,
            data_path= config.data_path,
            model_chkpt= config.model_chkpt,
            num_train_epochs= params.num_train_epochs,
            warmup_steps= params.warmup_steps,
            per_device_train_batch_size= params.per_device_train_batch_size,
            per_device_eval_batch_size= params.per_device_eval_batch_size,
            weight_decay= params.weight_decay,
            logging_steps= params.logging_steps,
            evaluation_strategy= params.evaluation_strategy,
            eval_steps= params.eval_steps,
            save_steps= params.save_steps,
            gradient_accumulation_steps= params.gradient_accumulation_steps
        )

        return model_trainer_config

In [23]:
import torch

from transformers import (Trainer, TrainingArguments,
                          DataCollatorForSeq2Seq,
                          AutoModelForSeq2SeqLM, AutoTokenizer)
from datasets import load_from_disk

from textSummarization.logging import logger

class ModelTrainer:
    def __init__(self, config:ModelTrainerConfig) -> None:
        self.config = config
    
    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        logger.info(f"Device using: {device}")
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_chkpt)
        model = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_chkpt).to(device)
        data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

        #Loading Data
        dataset = load_from_disk(self.config.data_path)

        trainer_args = TrainingArguments(
            output_dir= self.config.root_dir,
            num_train_epochs= self.config.num_train_epochs,
            warmup_steps= self.config.warmup_steps,
            per_device_train_batch_size= self.config.per_device_train_batch_size,
            per_device_eval_batch_size= self.config.per_device_eval_batch_size,
            weight_decay= self.config.weight_decay,
            logging_steps= self.config.logging_steps,
            evaluation_strategy= self.config.evaluation_strategy,
            eval_steps= self.config.eval_steps,
            save_steps= self.config.save_steps,
            gradient_accumulation_steps= self.config.gradient_accumulation_steps
        )

        trainer = Trainer(
            model= model,
            args= trainer_args,
            tokenizer= tokenizer,
            data_collator= data_collator,
            train_dataset= dataset["train"].select(list(range(1))),
            eval_dataset= dataset["validation"].select(list(range(1)))
        )
        "Using dataset.select(list(range(1))) just to test if everything is working as expected"

        trainer.train()

        #Save Model
        # trainer.save_model(save_directory=os.path.join(self.config.root_dir,"pegasus_samsum_model"))
        model.save_pretrained(save_directory=os.path.join(self.config.root_dir,"pegasus_samsum_model"))

        #Save Tokenizer
        tokenizer.save_pretrained(save_directory=os.path.join(self.config.root_dir,"tokenizer"))


In [24]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2024-03-19 23:47:09,400] 28 common - INFO - yaml file config\config.yml loaded successfully


[2024-03-19 23:47:09,418] 28 common - INFO - yaml file params.yml loaded successfully
[2024-03-19 23:47:09,433] 46 common - INFO - Created directory at: artifacts
[2024-03-19 23:47:09,444] 46 common - INFO - Created directory at: artifacts/model_trainer
[2024-03-19 23:47:09,451] 16 3859085255 - INFO - Device using: cpu


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\anfe1\OneDrive\Escritorio\carpetaANFE\AI_ML_Projects\Text-Summarization\venv\Lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
  0%|          | 0/1 [00:00<?, ?it/s]

: 